In [35]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing, cross_validation
import pandas as pd

In [36]:
df = pd.read_excel('titanic.xls')
print (df.head())

   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare    cabin embarked boat   body  \
0  29.0000      0      0   24160  211.3375       B5        S    2    NaN   
1   0.9167      1      2  113781  151.5500  C22 C26        S   11    NaN   
2   2.0000      1      2  113781  151.5500  C22 C26        S  NaN    NaN   
3  30.0000      1      2  113781  151.5500  C22 C26        S  NaN  135.0   
4  25.0000      1      2  113781  151.5500  C22 C26        S  NaN    NaN   

                         home.dest  
0                     St 

In [37]:
df.drop(['body','name'], 1, inplace=True)           #removing unnecessary columns
df.convert_objects(convert_numeric=True)
df.fillna(0, inplace=True)
print (df.head())

   pclass  survived     sex      age  sibsp  parch  ticket      fare    cabin  \
0       1         1  female  29.0000      0      0   24160  211.3375       B5   
1       1         1    male   0.9167      1      2  113781  151.5500  C22 C26   
2       1         0  female   2.0000      1      2  113781  151.5500  C22 C26   
3       1         0    male  30.0000      1      2  113781  151.5500  C22 C26   
4       1         0  female  25.0000      1      2  113781  151.5500  C22 C26   

  embarked boat                        home.dest  
0        S    2                     St Louis, MO  
1        S   11  Montreal, PQ / Chesterville, ON  
2        S    0  Montreal, PQ / Chesterville, ON  
3        S    0  Montreal, PQ / Chesterville, ON  
4        S    0  Montreal, PQ / Chesterville, ON  


C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [38]:
# a function to handle all non-numeric data from the dataset 
def handle_non_numerical_data(df):
    columns = df.columns.values                    #all the columns' name in 'columns variable'
    for column in columns:
        text_digit_vals = {}                       #a dictionary to store the column-values as numerical
        
        def convert_to_int(val):                   #a function to store the unique values in the dictionary
            return text_digit_vals[val]
        
                                                   #storing the uniques values inside each column and 
                                                   #giving a value to be stored in the dictionary
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist() #taking whole column as a list
            unique_elements = set(column_contents)       #finding how many unique values in each column
            x = 0
            for unique in unique_elements:               #giving each unique value a number and storing in the dict
                if unique not in text_digit_vals: 
                    text_digit_vals[unique] = x
                    x += 1
            
            df[column] = list(map(convert_to_int, df[column]))  #mapping using the funtion convert_to_int() 
    return df

In [39]:
df = handle_non_numerical_data(df)
print(df.head())

   pclass  survived  sex      age  sibsp  parch  ticket      fare  cabin  \
0       1         1    1  29.0000      0      0     747  211.3375    176   
1       1         1    0   0.9167      1      2     531  151.5500     96   
2       1         0    1   2.0000      1      2     531  151.5500     96   
3       1         0    0  30.0000      1      2     531  151.5500     96   
4       1         0    1  25.0000      1      2     531  151.5500     96   

   embarked  boat  home.dest  
0         3     2        271  
1         3    16          7  
2         3     0          7  
3         3     0          7  
4         3     0          7  


## so eikhane dekha jacche j shob column'gulai converted into numeric values unique numbers for each value in each column
    e.g: sex column'e : female=0, male=1 eivbe pura column'k convert kroa hoise

In [40]:
#try kore kore dekha jay, which column has how much effect on clustering accuracy
df.drop(['boat','sex'], 1, inplace=True)
print (df.head())

   pclass  survived      age  sibsp  parch  ticket      fare  cabin  embarked  \
0       1         1  29.0000      0      0     747  211.3375    176         3   
1       1         1   0.9167      1      2     531  151.5500     96         3   
2       1         0   2.0000      1      2     531  151.5500     96         3   
3       1         0  30.0000      1      2     531  151.5500     96         3   
4       1         0  25.0000      1      2     531  151.5500     96         3   

   home.dest  
0        271  
1          7  
2          7  
3          7  
4          7  


In [41]:
X = np.array(df.drop(['survived'], 1).astype(float))   #X is all but the survival column,and converted all to float datatypte
X = preprocessing.scale(X)                             #scaling X just to improve the accuracy
y = np.array(df['survived'])

In [42]:
clf = KMeans(n_clusters=2)                 #defining the classfier
clf.fit(X)                                 #trained the classifer with X

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [43]:
correct = 0
for i in range(len(X)):
    ##
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
    ## 
    ##we donot need to do these 3 lines in supervised learning coz, we wud know the ans b4 (see copy for more)
    if prediction[0] == y[i]:             #jodi prediction thik hoy, correctness baralum
        correct += 1

print (correct/len(X))                    #the accuracy of the clustering

0.6867838044308633


### accuracy will be reciprocal...i.e once it comes 70% and then it may come 30% and vice versa, etc. coz there r 2 clusters alwys. So this is just random from which cluster's view, the accuracy is shown.....see copy